# 📌 Guide d'Utilisation du Décodeur ASK/FSK

Ce programme permet de **décoder des fichiers modulés en ASK ou FSK** sous forme de signaux audio ou de données binaires. Il prend en charge les fichiers **CSV et WAV**, applique la **démodulation**, le **décodage Manchester**, la **vérification CRC**, et extrait les **données ASCII**.

## 🛠️ **Prérequis**
Avant d'exécuter le script, assurez-vous d'avoir installé les bibliothèques suivantes :

```bash
pip install numpy matplotlib scipy
```

## 🚀 **Exécution du Programme**
1. **Lancez le script** dans un environnement **Jupyter Notebook** ou un terminal Python.
2. **Sélectionnez le type de fichier** à décoder :
   - `1` pour un fichier **CSV** (texte modulé en ASK/FSK).
   - `2` pour un fichier **WAV** (signal audio modulé en ASK/FSK).
3. **Entrez le nom du fichier** (ex: `Message_ASK.wav` ou `Message_FSK.csv`).
4. **Choisissez la modulation** :
   - `ASK` pour **Amplitude Shift Keying**
   - `FSK` pour **Frequency Shift Keying**
5. Le programme **démodule, décode et affiche le message ASCII**.

## 📂 **Exemple d'Utilisation**
Si vous avez un fichier nommé `Message_ASK.wav`, suivez ces étapes :

```plaintext
Choisissez le type de fichier à décoder :
1 - Décoder un fichier texte (CSV)
2 - Décoder un fichier audio (WAV)
> 2
Entrez le nom du fichier WAV : Message_ASK.wav
Entrez le type de modulation (ASK ou FSK) : ASK
```

Résultat attendu :
```plaintext
Message décodé : Bonjour Professeur !
```

## 📊 **Affichage des Signaux**
Le programme génère **des graphiques des signaux** modulés et démodulés pour analyse visuelle.

## 📌 **Remarque**
- **Le fichier doit être dans le même répertoire** que le script.
- **La modulation choisie doit correspondre à celle utilisée pour l'encodage**.

---

🛠 **Développé pour le projet de Traitement du Signal** 🎛


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read
import os

###############################################################################
#                   PARAMÈTRES GLOBAUX
###############################################################################
Fe   = 44100        # Fréquence d'échantillonnage (Hz)
baud = 200          # Débit binaire (bits/s)
Ns   = int(Fe/baud) # Nombre d'échantillons par bit (~220)
Fp_ASK = 2000       # Fréquence de la porteuse ASK
fp1 = 200           # Fréquence FSK bit=1
fp2 = 3000          # Fréquence FSK bit=0

###############################################################################
#                   FONCTIONS DE DÉMODULATION & DÉCODAGE
###############################################################################

def demodulate_ASK(signal, Fe_used):
    """Démodule un signal ASK."""
    N = len(signal)
    t = np.arange(N) / Fe_used
    porteuse = np.sin(2 * np.pi * Fp_ASK * t)
    produit  = signal * porteuse
    bits = [1 if np.trapz(produit[i:i+Ns]) > 0 else 0 for i in range(0, N, Ns)]
    return bits

def demodulate_FSK(signal, Fe_used):
    """Démodule un signal FSK en comparant les résultats des porteuses fp1 et fp2."""
    N = len(signal)
    t = np.arange(N) / Fe_used
    s1 = np.sin(2 * np.pi * fp1 * t)
    s2 = np.sin(2 * np.pi * fp2 * t)
    prod1 = signal * s1
    prod2 = signal * s2
    bits = [1 if np.trapz(prod1[i:i+Ns]) > np.trapz(prod2[i:i+Ns]) else 0 for i in range(0, N, Ns)]
    return bits

def Manchester_decode(bits):
    """Décodage Manchester : (1,0) -> 1 et (0,1) -> 0."""
    return [1 if bits[i] == 1 and bits[i+1] == 0 else 0 for i in range(0, len(bits)-1, 2)]

def trame_reception(bits):
    """Extraction de la trame avec fanions, protocole, et données+CRC."""
    if len(bits) < 16:
        return None, None
    fanion_debut, fanion_fin = bits[:8], bits[-8:]
    if fanion_debut != [1]*8 or fanion_fin != [0]*8:
        return None, None
    core, protocole, data_crc = bits[8:-8], bits[8:16], bits[16:-8]
    type_fic = "txt" if protocole == [1,1,0,0,0,0,1,1] else "?"
    return data_crc, type_fic

def crcreception(bits):
    """Vérification CRC (bloc de 12 bits : 8 data + 4 CRC)."""
    diviseur = int("1001", 2)
    data_res, integrite_ok = [], True
    for i in range(0, len(bits), 12):
        if i+12 > len(bits): break
        data_8, reste_4 = bits[i:i+8], bits[i+8:i+12]
        reste_calc = int(''.join(map(str, data_8)) + "0000", 2) % diviseur
        if format(reste_calc, '04b') != ''.join(map(str, reste_4)):
            integrite_ok = False
        data_res.extend(data_8)
    return data_res if integrite_ok else None

def ASCII_decode(bits, type_fic):
    """Convertit une séquence binaire ASCII en texte."""
    if type_fic != "txt" or not bits:
        return "[Protocole inconnu ou données vides]"
    return ''.join(chr(int(''.join(map(str, bits[i:i+8])), 2)) for i in range(0, len(bits), 8))

def decode_file(filename, modulation_type):
    """Charge et décode un fichier CSV ou WAV en fonction du type de modulation."""
    if not os.path.isfile(filename):
        print(f"Le fichier '{filename}' n'existe pas.")
        return
    ext = os.path.splitext(filename)[1].lower()
    if ext == '.csv':
        signal = np.genfromtxt(filename, delimiter=',')
        Fe_used = Fe
    elif ext == '.wav':
        Fe_used, data = read(filename)
        signal = data[:, 0] if data.ndim > 1 else data.astype(np.float32)
    else:
        print("Format non pris en charge (CSV ou WAV uniquement).")
        return
    
    if modulation_type == "ASK":
        bits_demod = demodulate_ASK(signal, Fe_used)
    elif modulation_type == "FSK":
        bits_demod = demodulate_FSK(signal, Fe_used)
    else:
        print("Type de modulation inconnu.")
        return
    
    bits_manch = Manchester_decode(bits_demod)
    data_crc, type_fic = trame_reception(bits_manch)
    if data_crc is None:
        print("Trame invalide.")
        return
    data_ok = crcreception(data_crc)
    if data_ok is None:
        print("Erreur CRC, message corrompu.")
        return
    message = ASCII_decode(data_ok, type_fic)
    print("Message décodé :", message)

# Choix de l'utilisateur
print("Choisissez le type de fichier à décoder :")
print("1 - Décoder un fichier texte (CSV)")
print("2 - Décoder un fichier audio (WAV)")
choice = input("Entrez 1 pour un fichier CSV (text) ou 2 pour un fichier WAV (son) : ")
if choice == "1":
    filename = input("Entrez le nom du fichier CSV : ")
elif choice == "2":
    filename = input("Entrez le nom du fichier WAV : ")
else:
    print("Choix invalide.")
    exit()

modulation_type = input("Entrez le type de modulation (ASK ou FSK) : ").upper()
decode_file(filename, modulation_type)